In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils import data

from tqdm.notebook import tqdm_notebook

import warnings
warnings.filterwarnings('ignore')

device = torch.device("mps" if torch.has_mps else "cpu")
print(device)
from itertools import product
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from aif360.algorithms.postprocessing import (CalibratedEqOddsPostprocessing,
                                              EqOddsPostprocessing,
                                              RejectOptionClassification)
from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector
random_state = 1
from sklearn.linear_model import LogisticRegression


mps


pip install 'aif360[AdversarialDebiasing]'


In [2]:
# read in data and split
X = torch.load('inputs/bfw_senet50_face_embeddings.pt').cpu()
y = torch.load('inputs/bfw_senet50_labels.pt').cpu()
df = pd.read_csv('inputs/bfw_senet50_df.csv')
gender = df['reference_gender']
df

,reference_identity,candidate_identity,reference_ethnicity,candidate_ethnicity,reference_gender,candidate_gender,labels
0,n004721,n004721,asian,asian,male,male,1.0
1,n004721,n004721,asian,asian,male,male,1.0
2,n004721,n004721,asian,asian,male,male,1.0
3,n004721,n004721,asian,asian,male,male,1.0
4,n004721,n004721,asian,asian,male,male,1.0
...,...,...,...,...,...,...,...
38395,n003412,n005685,white,white,male,male,0.0
38396,n008917,n008839,black,black,female,female,0.0
38397,n008452,n008150,asian,asian,male,male,0.0
38398,n004936,n002509,asian,asian,female,female,0.0


In [5]:
train_split, test_split = train_test_split(np.arange(len(X)),test_size=0.2, random_state=random_state)
train_split, val_split = train_test_split(train_split,test_size=0.25, random_state=random_state)
X_train = X[train_split]
X_val = X[val_split]
X_test = X[test_split]
y_train = y[train_split]
y_val = y[val_split]
y_test = y[test_split]

gender_train = gender[train_split].values
gender_train[gender_train=='male'] = 0
gender_train[gender_train=='female'] = 1
gender_train = gender_train.astype(int)

gender_val = gender[val_split].values
gender_val[gender_val=='male'] = 0
gender_val[gender_val=='female'] = 1
gender_val = gender_val.astype(int)

gender_test = gender[test_split].values
gender_test[gender_test=='male'] = 0
gender_test[gender_test=='female'] = 1
gender_test = gender_test.astype(int)
train_df = df.iloc[train_split]
val_df = df.iloc[val_split]
test_df = df.iloc[test_split]
test_df

,reference_identity,candidate_identity,reference_ethnicity,candidate_ethnicity,reference_gender,candidate_gender,labels
340,n005178,n005178,asian,asian,male,male,1.0
18908,n001229,n001229,white,white,female,female,1.0
37816,n008972,n003463,asian,asian,male,male,0.0
10524,n000138,n000138,asian,asian,female,female,1.0
7492,n003398,n003398,white,white,male,male,1.0
...,...,...,...,...,...,...,...
7045,n007184,n007184,indian,indian,male,male,1.0
10213,n004472,n004472,asian,asian,female,female,1.0
12000,n002274,n002274,black,black,female,female,1.0
36114,n000660,n008636,white,white,female,female,0.0


In [6]:
def confusion_mat(y_pred, y_test):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    acc = (tn + tp)/(tn+tp+fn+fp)
    return tn, fp, fn, tp, acc
    
def AOE(tn_1,fp_1,fn_1,tp_1,tn_0,fp_0,fn_0,tp_0):
    tpr_1 = tp_1/(tp_1+fn_1)
    tpr_0 = tp_0/(tp_0+fn_0)

    fpr_1 = fp_1/(fp_1+tn_1)
    fpr_0 = fp_0/(fp_0+tn_0)


    return (np.abs(fpr_1-fpr_0) + np.abs(tpr_1 - tpr_0))/2

In [7]:
logisticRegr = LogisticRegression(solver = 'lbfgs')
logisticRegr.fit(X_train, y_train)
val_predictions = logisticRegr.predict(X_val)
val_predictions = np.array(val_predictions).flatten()
val_df['predictions'] = val_predictions
val_df

,reference_identity,candidate_identity,reference_ethnicity,candidate_ethnicity,reference_gender,candidate_gender,labels,predictions
32083,n006534,n002406,asian,asian,male,male,0.0,1.0
3576,n002241,n002241,black,black,male,male,1.0,0.0
22988,n000931,n002050,black,black,male,male,0.0,1.0
29398,n008322,n008067,indian,indian,male,male,0.0,1.0
35659,n002102,n005907,indian,indian,female,female,0.0,1.0
...,...,...,...,...,...,...,...,...
5141,n003243,n003243,indian,indian,male,male,1.0,1.0
27364,n009033,n000964,white,white,male,male,0.0,1.0
1022,n003463,n003463,asian,asian,male,male,1.0,0.0
5167,n006352,n006352,indian,indian,male,male,1.0,1.0


In [8]:
male_val_df = val_df[val_df['reference_gender']=='male']
female_val_df = val_df[val_df['reference_gender']=='female']

male_val_preds = male_val_df.predictions
female_val_preds = female_val_df.predictions

male_val_labels = male_val_df.labels
female_val_labels = female_val_df.labels


male_tn, male_fp, male_fn, male_tp, male_acc = confusion_mat(male_val_preds,male_val_labels)
female_tn, female_fp, female_fn, female_tp, female_acc = confusion_mat(female_val_preds,female_val_labels)


male_tnr = male_tn/(male_tn+male_fp)
male_tpr = male_tp/(male_tp+male_fn)
male_fnr = male_fn/(male_fn+male_tp)
male_fpr = male_fp/(male_fp+male_tn)
male_balanced_accuracy = (male_tpr + male_tnr)/2

female_tnr = female_tn/(female_tn+female_fp)
female_tpr = female_tp/(female_tp+female_fn)
female_fnr = female_fn/(female_fn+female_tp)
female_fpr = female_fp/(female_fp+female_tn)
female_balanced_accuracy = (female_tpr + female_tnr)/2

print(f'male TNR: {male_tn/(male_tn+male_fp):.3f} | male TPR: {male_tp/(male_tp+male_fn):.3f} ')
print(f'male FNR: {male_fn/(male_fn+male_tp):.3f} | male FPR: {male_fp/(male_fp+male_tn):.3f} ')
print(f'male ACC: {male_acc:.3f} | male balanced ACC: {male_balanced_accuracy:.3f} ')

print(f'female TNR: {female_tn/(female_tn+female_fp):.3f} | female TPR: {female_tp/(female_tp+female_fn):.3f} ')
print(f'female FNR: {female_fn/(female_fn+female_tp):.3f} | female FPR: {female_fp/(female_fp+female_tn):.3f} ')
print(f'female ACC: {female_acc:.3f} | female balanced ACC: {female_balanced_accuracy:.3f} ')

male TNR: 0.437 | male TPR: 0.487 
male FNR: 0.513 | male FPR: 0.563 
male ACC: 0.462 | male balanced ACC: 0.462 
female TNR: 0.449 | female TPR: 0.462 
female FNR: 0.538 | female FPR: 0.551 
female ACC: 0.455 | female balanced ACC: 0.456 
